# Build the structured box base model

## Notebook Setup

In [ ]:
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pathlib as pl
from shapely.geometry import Polygon, LineString
import flopy
from flopy.discretization import StructuredGrid

In [ ]:
from defaults import *
from plot_functions import *

In [ ]:
horiz_k = flopy.utils.Raster.load("./grid_data/box_k.asc")

### Parallel settings

1. Set `path_to_mf6` to the path of parallel MODFLOW 6 (`path\to\mf6`) if it is not in your `PATH` otherwise set to `None`.

In [ ]:
path_to_mf6 = pl.Path.home() / ".local/bin/mf6"

# Basin Structured Example

Structured grid parameters

In [ ]:
box_x, box_y = 19225.0, 19850.0
dx = dy = 100.0
nrow = int(box_y / dy) + 1
ncol = int(box_x / dx) + 1

Create a structured grid

In [ ]:
working_grid = StructuredGrid(
    nlay=1,
    delr=np.full(ncol, dx),
    delc=np.full(nrow, dy),
    xoff=0.0,
    yoff=0.0,
    top=np.full((nrow, ncol), 1000.0),
    botm=np.full((1, nrow, ncol), -100.0),
)

print(f"nrow: {nrow} ncol: {ncol}")

Sample topography

In [ ]:
npf_k = horiz_k.resample_to_grid(
    working_grid,
    band=horiz_k.bands[0],
    method="linear",
    extrapolate_edges=True,
)

In [ ]:
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
pmv = flopy.plot.PlotMapView(modelgrid=working_grid)
ax.set_aspect("equal")
v = pmv.plot_array(
    npf_k,
    norm=colors.LogNorm(vmin=npf_k.min(), vmax=npf_k.max()),
)
plt.colorbar(v, shrink=0.5)
ax.set_title("Sampled Hydraulic Coductivity");

### Build the base model

Set the simulation workspace

In [ ]:
list_valid_simulations()

In [ ]:
sim_ws = get_base_workspace(simulation_type="box_structured")
shutil.rmtree(sim_ws, ignore_errors=True)
sim_ws

Define the number of layers

In [ ]:
nlay = 1

Create the TDIS data

In [ ]:
nper = 3
tdis_data = [
    (1.0, 1, 1.0),
    (1000.0, 10, 1.5),
    (1.0, 1, 1.0),
]

Create the chd data

In [ ]:
chd_dict = {0: 48, ncol - 1: 40}
chd_data = []
for key, value in chd_dict.items():
    for row in range(nrow):
        chd_data.append((0, row, key, value))
chd_data[:5], chd_data[-5:]

Create the well data

In [ ]:
well_x, well_y = box_x / 2.0, box_y / 2.0
rw, cw = working_grid.intersect(well_x, well_y)
well_data = {
    1: [
        (0, rw, cw, -1000),
    ],
}
well_data

Create starting head data

In [ ]:
strt = list(chd_dict.values())[0]
strt

### Build the model files using FloPy

In [ ]:
sim = flopy.mf6.MFSimulation(
    sim_ws=sim_ws,
    exe_name="mf6",
    memory_print_option="summary",
)

tdis = flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_data)
ims = flopy.mf6.ModflowIms(
    sim,
    complexity="simple",
    print_option="SUMMARY",
    linear_acceleration="bicgstab",
    outer_maximum=1000,
    inner_maximum=100,
    outer_dvclose=1e-5,
    inner_dvclose=1e-6,
    preconditioner_levels=2,
    relaxation_factor=0.0,
)
gwf = flopy.mf6.ModflowGwf(
    sim,
    print_input=True,
    save_flows=True,
    newtonoptions="NEWTON UNDER_RELAXATION",
)

dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=dx,
    delc=dy,
    top=100.0,
    botm=-100.0,
    idomain=1,
    xorigin=0.0,
    yorigin=0.0,
)

ic = flopy.mf6.ModflowGwfic(gwf, strt=strt)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    icelltype=1,
    k=npf_k,
)
sto = flopy.mf6.ModflowGwfsto(
    gwf,
    sy=0.1,
    ss=1e-5,
    iconvert=1,
    steady_state={0: True, 2: True},
    transient={1: True},
)
chd = flopy.mf6.ModflowGwfchd(
    gwf,
    stress_period_data=chd_data,
)
wel = flopy.mf6.ModflowGwfwel(
    gwf,
    stress_period_data=well_data,
)
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{gwf.name}.hds",
    budget_filerecord=f"{gwf.name}.cbc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("BUDGET", "ALL")],
)

Count the number of active cells

In [ ]:
total_cells, active_cells = get_simulation_cell_count(sim)
total_cells, active_cells

### Write the model files

In [ ]:
sim.write_simulation()

In [ ]:
write_petscdb(sim_ws)

### Run the model

In [ ]:
if local_simulation():
    if path_to_mf6 is not None:
        sim.exe_name = path_to_mf6
    sim.run_simulation(processors=1)